In [ ]:
#!/usr/local/bin/python3

from secrets import randbits
from Crypto.Util.number import isPrime


class LCG:

    def __init__(self, a: int, c: int, m: int, seed: int):
        self.a = a
        self.c = c
        self.m = m
        self.state = seed

    def next(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state


while True:
    a = randbits(512)
    c = randbits(512)
    m = 1 << 512
    seed = randbits(512)
    initial_iters = randbits(16)
    # https://en.wikipedia.org/wiki/Linear_congruential_generator#m_a_power_of_2,_c_%E2%89%A0_0
    if (c != 0 and c % 2 == 1) and (a % 4 == 1):
        print(f"LCG coefficients:\na={a}\nc={c}\nm={m}")
        break

L = LCG(a, c, m, seed)
for i in range(initial_iters):
    L.next()

P = []
while len(P) < 2:
    test = L.next()
    if isPrime(test):
        P.append(test)

p, q = P

n = p * q

t = (p - 1) * (q - 1)

e = 65537

d = pow(e, -1, t)

message = int.from_bytes(open("flag.txt", "rb").read(), "big")

ct = pow(message, e, n)

print(f"n={n}")
print(f"ct={ct}")


In [ ]:
a=13151179517300634093331493823984335798104792889980233573449798803731216925285684304810858462637416538037295408686396679174318595674070448687202659818612957
c=5608153026248530688621721376357106253096775425754952602552939576813557496022590474668931015477737599864822088539029905392244325321894733034360048621379261
m=13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006084096
n=50101403006077774310803749857866737406441306821671558625795539694151977217751247484591812780124411136959890358141092389817206221245507789483550762690033519540724520476985696112416345603789409212360534931815255013428386819928723358410192823980871615952571298535642181672789853787284654429166275786540430857097
ct=45368552609653067612057824916369348133101260470242648823372346991953079959779269610598757268951375720246006930636033830813355756375343927330686424585808875611744608170167216612760800890947494428859402388924665165607428295669535846022365742274248308222529188860466330915820027097514123010732929776548001472954

In [ ]:
from Crypto.Util.number import isPrime, inverse
from sympy import symbols, solve, Poly
from sympy.ntheory.modular import crt

# 假设已经从输出中获取了这些值
a=13151179517300634093331493823984335798104792889980233573449798803731216925285684304810858462637416538037295408686396679174318595674070448687202659818612957
c=5608153026248530688621721376357106253096775425754952602552939576813557496022590474668931015477737599864822088539029905392244325321894733034360048621379261
m=13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006084096
n=50101403006077774310803749857866737406441306821671558625795539694151977217751247484591812780124411136959890358141092389817206221245507789483550762690033519540724520476985696112416345603789409212360534931815255013428386819928723358410192823980871615952571298535642181672789853787284654429166275786540430857097
ct=45368552609653067612057824916369348133101260470242648823372346991953079959779269610598757268951375720246006930636033830813355756375343927330686424585808875611744608170167216612760800890947494428859402388924665165607428295669535846022365742274248308222529188860466330915820027097514123010732929776548001472954
e = 65537

def find_primes():
    for k in range(1, 10000):  # 尝试不同的 k 值
        A = pow(a, k, m)
        B = 0
        for j in range(k):
            B = (B + pow(a, j, m) * c) % m
        # 构建二次方程 n = x * (A * x + B) mod m
        x = symbols('x')
        equation = Poly(x * (A * x + B) - n, x)
        solutions = solve(equation, x)
        for sol in solutions:
            if sol.is_integer:
                p_candidate = int(sol) % m
                if isPrime(p_candidate):
                    q_candidate = (A * p_candidate + B) % m
                    if isPrime(q_candidate) and p_candidate * q_candidate % m == n:
                        return p_candidate, q_candidate
    return None, None

p, q = find_primes()
if p and q:
    t = (p - 1) * (q - 1)
    d = inverse(e, t)
    message = pow(ct, d, n)
    flag = message.to_bytes((message.bit_length() + 7) // 8, "big")
    print(f"Decrypted flag: {flag.decode()}")
else:
    print("Failed to find primes.")